## Just Run, Dont Read

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM,LlamaTokenizer
import gc


from utils.PatchingMetric import ave_logit_diff
import torch
from utils.utils import load_dataset
from dataset.GreaterThan import GreaterThan

/mnt/lustre/work/eickhoff/esx670/.conda/py-311-pytorch_clone1/lib/python3.11/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/mnt/lustre/work/eickhoff/esx670/.conda/py-311-pytorch_clone1/lib/python3.11/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
from transformer_lens import utils, HookedTransformer, ActivationCache

def get_hooked_llm(model, device, task=""):
    model= HookedTransformer.from_pretrained(
        model_name=model,
        center_unembed=False,                    # set mean of every output vector W_U to zero
        center_writing_weights=False,            # normalize all input writing to residual stream
        fold_ln=False,                           # regularisation method: Hooked Transformer handels centering & normalization & linear operations all together, factor out te linear part
                                                # almost linear map: variance scalling divides by norm of vector -> norm not linear
        refactor_factored_attn_matrices=False,   # use low-rank matrices W_OV, W_QK instead of W_O and W_V (W_Q and W_K)
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True,
        )
    #model.set_use_hook_mlp_in(True)        
    model = model.to(device)   
    model.set_use_attn_result(True)
    model.set_use_split_qkv_input(True)  

    return model

In [6]:
gc.collect()
torch.cuda.empty_cache()
device = "cpu"
#device = "cuda:0"
N = 2
model = get_hooked_llm("gpt2", device)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("gpt2", force_download=False, use_fast=False)

/mnt/lustre/work/eickhoff/esx670/.conda/py-311-pytorch_clone1/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/mnt/lustre/work/eickhoff/esx670/.conda/py-311-pytorch_clone1/lib/python3.11/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cpu


## top k predictions

In [ ]:
def top_k_pred(model, dataset, tokenizer)

    outputs_clean = model(dataset.clean_tokens).logits
    outputs_corrupted = model(dataset.corrupted_tokens).logits


    for i in range(N):
        target_idx = dataset.target_idx[i]

        print("############ CLEAN INPUT ####################")
        next_token_logits = outputs_clean[target_idx[0], target_idx[1], :]

        k = 10
        top_k_values, top_k_indices = torch.topk(next_token_logits, k, largest=True)

        # Decode the predicted token IDs
        top_predicted_tokens = [tokenizer.decode([token_id]) for token_id in top_k_indices]

        # Display predictions
        print("input sentece:", tokenizer.batch_decode(dataset.clean_tokens[i, :target_idx[1] + 1]))
        print("top 10 answers")
        for rank, (token, score) in enumerate(zip(top_predicted_tokens, top_k_values), start=1):
            print(f"{rank}: {repr(token)} (score: {score.item():.4f})")
            

        print("############ CORRUPTED INPUT ###############")
        next_token_logits = outputs_corrupted[target_idx[0], target_idx[1], :]

        k = 10
        top_k_values, top_k_indices = torch.topk(next_token_logits, k, largest=True)

        # Decode the predicted token IDs
        top_predicted_tokens = [tokenizer.decode([token_id]) for token_id in top_k_indices]

        # Display predictions
        print("input sentece:", tokenizer.batch_decode(dataset.corrupted_tokens[i, :target_idx[1]+1]))
        print("top 10 answers")
        for rank, (token, score) in enumerate(zip(top_predicted_tokens, top_k_values), start=1):
            print(f"{rank}: {repr(token)} (score: {score.item():.4f})")
        print()
            


In [ ]:
from utils.utils import print_whole_dataset

## Greater Than

In [115]:
dataset = load_dataset(prepend_bos=False, task="ioi", patching_method="path",tokenizer=tokenizer, N=N, device=device)

device cpu


### Target Labels equals Answer Labels?

In [202]:
for n in range(dataset.N):
    print("clean token: \n", dataset.clean_input[n])
    print("correct answers: \n", tokenizer.batch_decode(dataset.answer_tokens[0][n]))
    print("wrong answers: \n", tokenizer.batch_decode(dataset.answer_tokens[1][n]))
    print()

clean token: 
 The attack lasted from the year 1566 to the year 15
correct answers: 
 ['67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '

### Logit Difference

In [163]:
with torch.no_grad():
    logits_gt = model(dataset.clean_dataset.toks)
    
ave_logit_gt = ave_logit_diff(logits_gt.to(device), dataset.answer_tokens, dataset.target_idx.to(device))
print("Average logit difference for GreaterThan task: ", ave_logit_gt)

Average logit difference for GreaterThan task:  4.0012359619140625


#### Test Input

In [ ]:
logits = torch.Tensor([
    [[-100, -100, -100, -100, -100, -100],
    [1, 2, 3, 4, 5, 6]],
    [[-100, -100, -100, -100, -100, -100],
    [7, 8, 9, 10, 11, 12]]
    ])
answer_tokens = {"correct": [ torch.tensor([0, 1], dtype=torch.int64), torch.tensor([0, 1, 2], dtype=torch.int64)],
                    "wrong": [torch.tensor([4, 5], dtype=torch.int64), torch.tensor([3, 4, 5], dtype=torch.int64)]}

ave_logit_test = ave_logit_diff(logits.to(device), answer_tokens, None)
print("Average logit difference for test task: ", ave_logit_test)

corr = [(1 + 2)/2 -(5 + 6)/2, (7 + 8 + 9)/3 - (10 + 11 + 12)/3]
print("expected result:", torch.mean(torch.tensor(corr)).item())



correct answer tensor([0, 1])
tensor([[1., 2.]])
tensor([[1., 2.]])
wrong answer tensor([[5., 6.]])
correct answer tensor([0, 1, 2])
tensor([[1., 2., 3.]])
tensor([[7., 8., 9.]])
wrong answer tensor([[10., 11., 12.]])
right prediction logits tensor([1.5000, 8.0000])
wrong prediction logits tensor([ 5.5000, 11.0000])
Average logit difference for test task:  -3.5
expected result: -3.5


## Induction

In [76]:
N = 2
from dataset.NameInduction import *
tokenizer.pad_token = tokenizer.eos_token
dataset = NameInduction(N, tokenizer, device,1234, prepend_bos=True)


length of names 39


## Gendered Pronoun

In [101]:
from dataset.GenderedPronoun import GenderedPronoun

N = 20
dataset = GenderedPronoun(
    N= N, 
    tokenizer=tokenizer, 
    device=device, 
    seed=1234, 
    prepend_bos=True
    )

John [1757]
David [3271]
Mark [2940]
Paul [3362]
Ryan [6047]
Gary [10936]
Jack [3619]
Sean [11465]
Carl [8124]
Joe [5689]
Mary [5335]
Lisa [15378]
Anna [11735]
Sarah [10490]
Amy [14235]
Carol [5074]
Karen [18678]
Susan [14919]
Julie [21946]
Judy [26395]
Pat [3208]
Kit [10897]
Sky [5274]
Lux [17145]
Ash [7844]
['Pat', 'Kit', 'Sky', 'Lux', 'Ash']


## Docstring